# Bangkok Tourists Recommender System
### Written by Soravis Prommas

## Table of contents

* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction <a name="introduction"></a>

Bangkok is one of the world's most visited cities. The city is considerably famous among tourists from every corner of the world. There are large number of tourists that have frequently been to Bangkok and visited alomost every of its neighborhoods. While another number of tourists also desire to explore this amazing experience, but they don't get any idea about where to visit and what to do in Bangkok.

The purpose of this project is to help those new tourists in exploring the venues, activities and facilities around the city. Since each area has different tone and characteristic, it will help people making smart and efficient decisions on visiting some great neighborhoods matching their interests and lifestyle out of the numbers of other neighborhoods in Bangkok.

This project aims to make an analysis and categorize the neighborhoods (districts) in Bangkok. The neighborhoods sharing some common groups of venues and places will be defined as the same category.

## 2. Data <a name="data"></a>

The neighborhood information will be scraped from https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok. The desired information will consist of the names of the districts and their coordinates (latitudes and longitudes). Then, the Foursquare API will be used to search for the venues in each district. Using an API's specific endpoint, information retrieved from the API will be a list of venue's names and their category around the given radius of the input latitude and longitude. For this project, the searching radius will be chosen as 1,000 meters since each area is quite large.

Based on definition of our problem, information that needs to be collected includes:
* List of neighhborhoods in Bangkok
* Location (latitude, longitude) of each neighborhood
* Existing venues in each neighborhood
* Venue categories

### Neighborhood Candidates

Firstly, let's scrape the neighborhood names and locations from a Wikipedia page.

In [5]:
# Import relevant libraries for web scraping
import pandas as pd
import requests
import geocoder
import folium
from bs4 import BeautifulSoup

In [6]:
# Retreive the http response
html=requests.get('https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok')
soup=BeautifulSoup(html.content, 'html.parser')

In [7]:
# Create a blank dataframe for storing scraped data
df_bangkok=pd.DataFrame(columns=['Neighborhood','Latitude','Longitude'])

table=soup.find('table', class_='wikitable sortable')

for i,tr in enumerate(table.find_all('tr')[1:]):
        tds=tr.find_all('td')
        # Put scapted data into the dataframe
        df_bangkok.loc[i,'Neighborhood']=tds[0].text.strip()
        df_bangkok.loc[i,'Latitude']=tds[6].text.strip()
        df_bangkok.loc[i,'Longitude']=tds[7].text.strip()

We now have a dataframe looks like below.

In [8]:
df_bangkok.head()

,Neighborhood,Latitude,Longitude
0,Bang Bon,13.6592,100.3991
1,Bang Kapi,13.765833,100.647778
2,Bang Khae,13.696111,100.409444
3,Bang Khen,13.873889,100.596389
4,Bang Kho Laem,13.693333,100.5025


### Neighborhood Exploration

In the next step, we will use a Foursqure API endpoint called "explore" to retrieve all venues availeble around the given cooridates. The API requires a latitude, a longitude and a searching radius as input parameters.

In [9]:
# Define Foursquare API credentials
CLIENT_ID = 'TZLDDXPIS4ZANAZRI5ZYQ0Q4FHBLPQMMVJ1MCZ3EOBOL2Q5B' 
CLIENT_SECRET = 'PWY3OSMGFI3CZIUMKWOZROAP30FJVT1TFHHKCFHRF3J4ATXV'
VERSION = '20180605'
LIMIT = 100

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
# Get information for the nearby venues
bangkok_venues = getNearbyVenues(names=df_bangkok['Neighborhood'],
                                   latitudes=df_bangkok['Latitude'],
                                   longitudes=df_bangkok['Longitude']
                                  )

In [12]:
bangkok_venues=bangkok_venues[bangkok_venues['Venue Category']!='Neighborhood']

We got all the venues and their categories inside each neighborhood.

In [13]:
bangkok_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bang Bon,13.6592,100.3991,ขาหมูบางหว้า,13.657136,100.395230,Thai Restaurant
1,Bang Bon,13.6592,100.3991,Irashaimase Japanese Restaurant,13.658358,100.401403,Japanese Restaurant
2,Bang Bon,13.6592,100.3991,ร้านต้นไม้ ริมถนนกาญจนาภิเษก,13.654098,100.405054,Garden Center
3,Bang Bon,13.6592,100.3991,เจ๊ไน้ ก๋วยเตี๋ยวเป็ดตุ๋น,13.654137,100.405323,Noodle House
4,Bang Bon,13.6592,100.3991,ส.สุขโภชนา,13.660358,100.404866,Chinese Restaurant


## 3. Methodology <a name="methodology"></a>

According to the problem definition, we want to group the similar neighborhoods together. To compare the similarities of neighborhoods, we explore neighborhoods, count the number of venues in each category and group them into clusters to find similar neighborhoods in Bangkok. To be able to do that, we need to cluster data by using a machine learning algorithm **K-means Clustering**.

## 4. Analysis <a name="analysis"></a>

In Section 2, we have a dataframe containing neighborhoods along with their venues and venue categories. We want to count the number of venues that a neighborhood has in each category.

Firstly, convert all category labels into columns.

In [14]:
# one hot encoding
bangkok_onehot = pd.get_dummies(bangkok_venues[['Venue Category']], prefix="", prefix_sep="")
cat_columns=bangkok_onehot.columns

# add neighborhood column back to dataframe
bangkok_onehot['Neighborhood'] = bangkok_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ['Neighborhood'] + list(cat_columns)
bangkok_onehot = bangkok_onehot[fixed_columns]

bangkok_onehot.head()

,Neighborhood,Accessories Store,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping rows by Neighborhood and by taking the sum of the frequency of occurrence of each category will give the number of venues of each category.

In [15]:
bangkok_grouped = bangkok_onehot.groupby('Neighborhood').sum().reset_index()
bangkok_grouped.head()

,Neighborhood,Accessories Store,Airport Service,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,Bang Bon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bang Kapi,0,0,0,0,0,0,0,0,1,...,0,0,1,1,1,0,0,1,0,0
2,Bang Khae,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Bang Khen,0,0,0,0,0,0,0,0,3,...,0,0,2,0,0,0,0,0,0,0
4,Bang Kho Laem,0,0,0,0,2,0,0,0,4,...,0,0,1,0,0,0,0,0,0,0


Create a new dataframe to see which venue categories are the most common of a neighborhood.

In [16]:
import numpy as np

In [17]:
# define a function to sort the category descendingly
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangkok_grouped['Neighborhood']

for ind in np.arange(bangkok_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangkok_grouped.iloc[ind, :], num_top_venues)

In [18]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,Noodle House,Convenience Store,Thai Restaurant,Coffee Shop,Shopping Mall,Chinese Restaurant,Bar,Café,Flower Shop,Som Tum Restaurant
1,Bang Kapi,Coffee Shop,Japanese Restaurant,Som Tum Restaurant,Noodle House,Thai Restaurant,Fast Food Restaurant,Clothing Store,Hotpot Restaurant,Ramen Restaurant,Dessert Shop
2,Bang Khae,Convenience Store,Noodle House,Japanese Restaurant,Shopping Mall,Shop & Service,Coffee Shop,Thai Restaurant,Bar,Fast Food Restaurant,Diner
3,Bang Khen,Convenience Store,Coffee Shop,Fast Food Restaurant,Bookstore,Steakhouse,Hotpot Restaurant,Thai Restaurant,Asian Restaurant,Noodle House,Ramen Restaurant
4,Bang Kho Laem,Coffee Shop,Noodle House,Convenience Store,Thai Restaurant,Asian Restaurant,Chinese Restaurant,Seafood Restaurant,Arcade,Pub,Hotel


Run _k_-means to cluster the neighborhood into 4 clusters.

In [19]:
from sklearn.cluster import KMeans

In [20]:
# set number of clusters
kclusters = 4

bangkok_grouped_clustering = bangkok_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangkok_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 0, 2, 0, 0, 0, 2, 2, 1, 3, 2, 3, 2, 0, 2, 2, 2, 3, 3, 0, 2, 1,
       2, 0, 2, 0, 2, 2, 2, 3, 0, 1, 0, 1, 3, 2, 0, 1, 2, 1, 2, 3, 3, 2,
       2, 2, 2, 0, 1, 0])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangkok_merged = df_bangkok

bangkok_merged = bangkok_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
bangkok_merged.reset_index(drop=True, inplace=True)
bangkok_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bang Bon,13.6592,100.3991,2,Noodle House,Convenience Store,Thai Restaurant,Coffee Shop,Shopping Mall,Chinese Restaurant,Bar,Café,Flower Shop,Som Tum Restaurant
1,Bang Kapi,13.765833,100.647778,0,Coffee Shop,Japanese Restaurant,Som Tum Restaurant,Noodle House,Thai Restaurant,Fast Food Restaurant,Clothing Store,Hotpot Restaurant,Ramen Restaurant,Dessert Shop
2,Bang Khae,13.696111,100.409444,2,Convenience Store,Noodle House,Japanese Restaurant,Shopping Mall,Shop & Service,Coffee Shop,Thai Restaurant,Bar,Fast Food Restaurant,Diner
3,Bang Khen,13.873889,100.596389,0,Convenience Store,Coffee Shop,Fast Food Restaurant,Bookstore,Steakhouse,Hotpot Restaurant,Thai Restaurant,Asian Restaurant,Noodle House,Ramen Restaurant
4,Bang Kho Laem,13.693333,100.5025,0,Coffee Shop,Noodle House,Convenience Store,Thai Restaurant,Asian Restaurant,Chinese Restaurant,Seafood Restaurant,Arcade,Pub,Hotel


Finally, let's visualize the resulting clusters

In [22]:
import folium
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors

In [23]:
# create map
map_clusters = folium.Map(location=[13.736717, 100.523186], zoom_start=10) # location of Bangkok is 13.736717, 100.523186

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangkok_merged['Latitude'], bangkok_merged['Longitude'], bangkok_merged['Neighborhood'], bangkok_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Result for Cluster 0:**

In [24]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster Labels']==0]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,0,Bang Kapi,Coffee Shop,Japanese Restaurant,Som Tum Restaurant,Noodle House,Thai Restaurant,Fast Food Restaurant,Clothing Store,Hotpot Restaurant,Ramen Restaurant,Dessert Shop
3,0,Bang Khen,Convenience Store,Coffee Shop,Fast Food Restaurant,Bookstore,Steakhouse,Hotpot Restaurant,Thai Restaurant,Asian Restaurant,Noodle House,Ramen Restaurant
4,0,Bang Kho Laem,Coffee Shop,Noodle House,Convenience Store,Thai Restaurant,Asian Restaurant,Chinese Restaurant,Seafood Restaurant,Arcade,Pub,Hotel
5,0,Bang Khun Thian,Thai Restaurant,Coffee Shop,Japanese Restaurant,Ice Cream Shop,Asian Restaurant,Restaurant,Pizza Place,Bar,Bookstore,Steakhouse
13,0,Chatuchak,Coffee Shop,Thai Restaurant,Noodle House,Fast Food Restaurant,Japanese Restaurant,Flea Market,Café,Bakery,Shabu-Shabu Restaurant,Gym / Fitness Center
19,0,Khan Na Yao,Coffee Shop,Japanese Restaurant,Noodle House,Ice Cream Shop,Bakery,Convenience Store,Pizza Place,Thai Restaurant,Som Tum Restaurant,Café
23,0,Lak Si,Coffee Shop,Convenience Store,Thai Restaurant,Fast Food Restaurant,Gym / Fitness Center,Bar,Flea Market,Gym,Steakhouse,Japanese Restaurant
25,0,Lat Phrao,Coffee Shop,Noodle House,Convenience Store,Som Tum Restaurant,Café,Asian Restaurant,Restaurant,Thai Restaurant,BBQ Joint,Seafood Restaurant
30,0,Phasi Charoen,Coffee Shop,Convenience Store,Thai Restaurant,BBQ Joint,Fast Food Restaurant,Steakhouse,Japanese Restaurant,Asian Restaurant,Pizza Place,Noodle House
32,0,Phra Khanong,Convenience Store,Coffee Shop,Hotel,Noodle House,Fast Food Restaurant,Ice Cream Shop,Italian Restaurant,Chinese Restaurant,Bakery,Spa


**Result for Cluster 1:**

In [25]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster Labels']==1]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,1,Bang Rak,Noodle House,Thai Restaurant,Hotel,Chinese Restaurant,Café,Seafood Restaurant,Coffee Shop,Bar,Spa,Japanese Restaurant
21,1,Khlong San,Coffee Shop,Noodle House,Chinese Restaurant,Dessert Shop,Café,Thai Restaurant,Restaurant,Art Gallery,Hotel Bar,Japanese Restaurant
31,1,Phaya Thai,Bar,Thai Restaurant,Café,Coffee Shop,Japanese Restaurant,Noodle House,Sushi Restaurant,Restaurant,Bakery,Som Tum Restaurant
33,1,Phra Nakhon,Thai Restaurant,Bar,Hotel,Café,Noodle House,Asian Restaurant,Massage Studio,Spa,Vegetarian / Vegan Restaurant,Hostel
37,1,Ratchathewi,Noodle House,Coffee Shop,Hotel,Café,Steakhouse,Hostel,Japanese Restaurant,Som Tum Restaurant,Thai Restaurant,Convenience Store
39,1,Samphanthawong,Thai Restaurant,Coffee Shop,Dessert Shop,Café,Noodle House,Art Gallery,Hotel,Hotel Bar,Bar,Seafood Restaurant
48,1,Watthana,Coffee Shop,Café,Japanese Restaurant,Thai Restaurant,Noodle House,Bar,Hotel,Shopping Mall,Shabu-Shabu Restaurant,Asian Restaurant


**Result for Cluster 2:**

In [26]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster Labels']==2]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Bang Bon,Noodle House,Convenience Store,Thai Restaurant,Coffee Shop,Shopping Mall,Chinese Restaurant,Bar,Café,Flower Shop,Som Tum Restaurant
2,2,Bang Khae,Convenience Store,Noodle House,Japanese Restaurant,Shopping Mall,Shop & Service,Coffee Shop,Thai Restaurant,Bar,Fast Food Restaurant,Diner
6,2,Bang Na,Thai Restaurant,Noodle House,Pier,Café,Asian Restaurant,Coffee Shop,Convenience Store,Seafood Restaurant,Supermarket,Market
7,2,Bang Phlat,Convenience Store,Coffee Shop,Fast Food Restaurant,Hotpot Restaurant,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Massage Studio,Sporting Goods Shop,Flea Market
10,2,Bangkok Noi,Noodle House,Som Tum Restaurant,Asian Restaurant,Convenience Store,Café,Coffee Shop,Food Truck,Dim Sum Restaurant,Dessert Shop,Mobile Phone Shop
12,2,Bueng Kum,Coffee Shop,Convenience Store,Café,Supermarket,Thai Restaurant,Bakery,Food Truck,Flea Market,Park,Furniture / Home Store
14,2,Chom Thong,Thai Restaurant,Convenience Store,Coffee Shop,Dog Run,Asian Restaurant,Bar,BBQ Joint,Pet Store,Food Truck,Fried Chicken Joint
15,2,Din Daeng,Convenience Store,Noodle House,Thai Restaurant,Sports Club,Restaurant,Chinese Restaurant,Stadium,Soccer Field,Burger Joint,Café
16,2,Don Mueang,Noodle House,Thai Restaurant,Convenience Store,Breakfast Spot,Seafood Restaurant,Flea Market,Bus Station,Restaurant,Chinese Restaurant,Buffet
20,2,Khlong Sam Wa,Exhibit,Restaurant,Coffee Shop,Thai Restaurant,Noodle House,Asian Restaurant,Convenience Store,Dessert Shop,Pub,Japanese Restaurant


**Result for Cluster 3:**

In [27]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster Labels']==3]

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,3,Bang Sue,Noodle House,Thai Restaurant,Convenience Store,Train Station,Coffee Shop,Ice Cream Shop,Asian Restaurant,Hotpot Restaurant,Restaurant,Som Tum Restaurant
11,3,Bangkok Yai,Noodle House,Asian Restaurant,Convenience Store,Farmers Market,BBQ Joint,Dessert Shop,Flea Market,Coffee Shop,Train Station,Dim Sum Restaurant
17,3,Dusit,Noodle House,Thai Restaurant,Convenience Store,Asian Restaurant,Café,Som Tum Restaurant,Coffee Shop,Palace,Chinese Restaurant,Italian Restaurant
18,3,Huai Khwang,Thai Restaurant,Noodle House,Convenience Store,Japanese Restaurant,Dessert Shop,Hotel,Som Tum Restaurant,Asian Restaurant,Pub,Café
29,3,Pathum Wan,Noodle House,Asian Restaurant,Seafood Restaurant,Hotel,Thai Restaurant,Dessert Shop,Chinese Restaurant,Gym / Fitness Center,Hostel,Ice Cream Shop
34,3,Pom Prap Sattru Phai,Noodle House,Café,Thai Restaurant,Convenience Store,Asian Restaurant,Chinese Restaurant,Hotel,Coffee Shop,Historic Site,Museum
41,3,Sathon,Noodle House,Asian Restaurant,Convenience Store,Chinese Restaurant,Thai Restaurant,Bar,Dessert Shop,Coffee Shop,Hotpot Restaurant,Hotel
42,3,Suan Luang,Noodle House,Coffee Shop,Convenience Store,Asian Restaurant,Thai Restaurant,Dessert Shop,Massage Studio,Som Tum Restaurant,Shopping Mall,Racetrack


## 5. Results and Discussion <a name="results"></a>

Looking at the results of 4 clusters, we may define each cluster as follows:

**Cluster 0:**
Obviously, this cluster is full of convenience stores, restaurants and cafe. People who love shopping are recommended to visit these neighborhoods. Every branded product in Thailand is available here.

**Cluster 1:**
Neighborhoods in this cluster seem to be the most well-known by the tourists especially Phra Nakhon which is the location of Khaosarn Road, a famous place to drink and enjoy night life. This cluster is full of pubs, bars and hotels. So, it's suitable for young backpackers who love party and entertainment. Not recommended for kids and elders.

**Cluster 2:**
This is the suburb of Bangkok. Although there may be lots of restaurant, but most of them all local street food stalls. Either way, the covenience stores here are not that good and most of them are small stores. You may not find every things you want here. Actually, many areas in this cluster are not tourists' destination even for Thai tourists.

**Cluster 3:**
This is the central Bangkok. Completely opposite to cluster 2. Many premium restaurants and five-star hotels are located here. This is also the center of business areas. It also provides the best facilities and infrastructure in Bangkok. Surely this cluster is suitable for tourists who love to try premium Thai and Asian foods and love to enjoy luxury life, not for the low-cost travelers at all. Aside from tourism, it is the best place to run any business.

It is to be noted that the definition above is from the result of our analysis using venue information from **Foursqaure** alone, which may not provide information as much as it does for neighborhoods in the U.S. or any other countries. 

## 6. Conclusion <a name="conclusion"></a>

Purpose of this project is to roughly define the characteristic of each neighborhood in Bangkok for the ease of tourists to establish their destinations and to create their traveling plans. We  gathered information about venues in the neighborhoods from Foursquare, then clustered the neighborhoods in regarding with their categories of the ten most common venues using K-mean algorithm. From the final result, we got four clusters and defined the characteristic for each cluster. For the further development of this project, more information should be gathered from sources other than Foursquare alone which may provide a better analysis.